# PromptGuard

[PromptGuard](https://github.com/opaque-systems/promptguard-python) preserves the privacy in your prompts.
 

This notebook goes over how to use LangChain to interact with `PromptGuard`.

In [ ]:
# install the promptguard and langchain packages
! pip install promptguard langchain

import os

# Set API keys
# Join the [waitlist](tbd) for the an API Key to access the promptguard API.

os.environ['PROMPT_GUARD_ACCESS_TOKEN'] = "<PROMPT_GUARD_API_KEY>"
os.environ['OPENAI_API_KEY'] = "<OPENAI_API_KEY>"

# Use PromptGuardLLMWrapper

Applying promptguard to your application could be as simple as wrapping your LLM using the PromptGuardLLMWrapper class. 
```python
# llm=OpenAI()
llm=PromptGuardLLMWrapper(OpenAI())
```

In [ ]:
import langchain
from langchain import LLMChain, PromptTemplate
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory

from langchain.llms import PromptGuardLLMWrapper

langchain.verbose = True
langchain.debug = True

prompt_template = """
As an AI assisant, you will answer questions according to given context.

Important PII data is sanitized in the question.
For example, "Giana is good" is sanitized to "PERSON_999 is good".
You must treat the sanitized data as opaque strings, but you can use them as
meaningful entities in the response.
Different sanitized items could be the same entity based on the semantics.
You must keep the sanitized item as is and cannot change it.
The format of sanitized item is "TYPE_ID".
You must not create new sanitized items following the format. For example, you
cannot create "PERSON_1000" or "PERSON_998" if "PERSON_1000" or "PERSON_998" is
not in the question.

Conversation History: ```{history}```
Context : ```Mr. Carl Smith is a 31-year-old man who has been experiencing
homelessness on and off for all his adult life. Mr. Smith says he is about
5’5” and weighs approximately 129 lbs. He presents as
very thin, typically wearing a clean white undershirt
and loose-fitting khaki shorts at interviews.
His brown hair is disheveled and dirty looking, and
he constantly fidgets and shakes his hand or
knee during interviews. Despite his best efforts, Carl is a poor historian. ```
Question: ```{question}```

"""

chain = LLMChain(
    prompt=PromptTemplate.from_template(prompt_template),
    llm=PromptGuardLLMWrapper(llm=OpenAI()),
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True,
)


print(
    chain.run(
        {"question": """How high is he? """},
        callbacks=[StdOutCallbackHandler()],
    )
)

From the output, you can see the following context from user input has sensitive data.

``` 
# Context from user input

Mr. Carl Smith is a 31-year-old man who has been experiencing
homelessness on and off for all his adult life. Mr. Smith says he is about
5’5” and weighs approximately 129 lbs. He presents as
very thin, typically wearing a clean white undershirt
and loose-fitting khaki shorts at interviews.
His brown hair is disheveled and dirty looking, and
he constantly fidgets and shakes his hand or
knee during interviews. Despite his best efforts, Carl is a poor historian. 
```

PromptGuard will automatically detect the sensitive data and replace it with a placeholder. 

```

# Context after PromptGuard

Mr. PERSON_5 is a DATE_TIME_1 man who has been experiencing
homelessness on and off for all his adult life. Mr. PERSON_4 says he is about
5’5” and weighs approximately 129 lbs. He presents as
very thin, typically wearing a clean white undershirt
and loose-fitting PERSON_3 at interviews.
His brown hair is disheveled and dirty looking, and
he constantly fidgets and shakes his hand or
knee during interviews. Despite his best efforts, PERSON_2 is a poor historian.
```

Placeholder is used in the LLM response.

```
# response returned by LLM
Mr. PERSON_5 is approximately 5'5" tall.
```

Response is desanitized by replacing the placeholder with the original sensitive data.

```
# desanitized LLM response from PromptGuard
Mr. Carl Smith is approximately 5'5" tall.
```

# Use promptguard in LangChain expression

There are functions that can be used with LangChain expression as well if a drop-in replacement doesn't offer the flexibility you need. 

In [ ]:
import langchain.utilities.promptguard as pgf
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser


prompt=PromptTemplate.from_template(prompt_template),    
llm = OpenAI()
pg_chain = (
    pgf.sanitize
    | RunnableMap(
        {
            "response": (lambda x: x["sanitized_input"])
            | prompt
            | llm
            | StrOutputParser(),
            "secure_context": lambda x: x["secure_context"],
        }
    )
    | (lambda x: pgf.desanitize(x["response"], x["secure_context"]))
)

pg_chain.invoke({"question": "How high is he?", "history": ""})